# Sentiment Analysis Using Tensorflow LSTM - CNN with Attention Layer

code below takes ages on dell lat 5401 (one epoch more than 9 hours, actual time unknown since I have quit after 9 hours) does not seem to be worthwhile unless run on supercomputer 

## Imports

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# from tensorflow.keras.models import Sequential, load_model
# from tensorflow.compat.v1.keras.layers import CuDNNLSTM, Embedding, Dropout,Dense, Bidirectional, BatchNormalization
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.optimizers import RMSprop, Adamax , Adam
# from tensorflow.keras import regularizers
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.preprocessing.sequence import pad_sequences

from attention.layers import AttentionLayer
import runai.elastic
import tensorflow as tf
from tensorflow.python.client import device_lib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #This is for multiple print statements per cell

import tensorflow.compat.v1
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
# from tensorflow.compat.v1.keras.layers import CuDNNLSTM
# from tensorflow.compat.v1.keras.layers import CuDNNLSTM, Embedding, Dropout,Dense, Bidirectional, BatchNormalization



config = ConfigProto()
config.allow_soft_placement=False
# config.gpu_options.per_process_gpu_memory_fraction = 0.95
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Using TensorFlow backend.


## Confirm GPU processing available

In [2]:
value = tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)
print ('***If TF can access GPU: ***\n\n',value) # MUST RETURN True IF IT CAN!!

print()
value = tf.config.list_physical_devices('GPU')
print(value)

print()
print(device_lib.list_local_devices())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
***If TF can access GPU: ***

 True

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17706593274254407271
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8011231295283081122
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16683770576981217268
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1501691904
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18049748788423014427
physical_device_desc: "device: 0, name: GeForce MX150, pci bus id: 0000:02:00.0, compute capability: 6.1"
]


## Getting data

In [3]:
df = pd.read_csv('consolidated_tweet_data-cleaned-stemmed-lemmatized.csv', sep='\t')
df

,sentiment,text
0,negative,oh no it fade away again
1,positive,bunnylak will kill me but i cant stop listen t...
2,negative,last day in cali partyin for the last time wit...
3,negative,is have a major soar throat
4,positive,my last day a 12 year old
...,...,...
1611536,negative,twisuz yeah and how did thi happen i wa updat ...
1611537,negative,smittygoali im sorri about your dog
1611538,negative,posipat im alreadi there i wish you were here
1611539,negative,is think in 12 hour ill be at the airport thi ...


In [4]:
# How much of Dataset to be used
frac = 0.5
# sample and shuffle the dataset according to the fraction choise in the line above
df1 = df.sample(frac=frac).reset_index(drop=True)
df1

,sentiment,text
0,negative,work 311
1,positive,giagia that when your tummi smile
2,negative,thatboygotsol i do but im not by a comp
3,negative,watch who line with my dad geesu i miss thi show
4,negative,dougiemcfli hiya im a big fan but im reali bor...
...,...,...
805765,positive,watch show i miss on the ellipit love my ondem...
805766,positive,nickytvf dalla what go on in dalla im in dalla
805767,negative,beautifulwreck2 veri sorri to hear that theonl...
805768,negative,alanodea i couldnt find a way to regist for be...


## Tokenization for training

In [5]:
vocabulary_size = 100000

In [6]:
tokenizer = Tokenizer(num_words=vocabulary_size, split=" ", oov_token='<OOV>')
tokenizer.fit_on_texts(df['text'].values)

In [7]:
X = tokenizer.texts_to_sequences(df1['text'].values)
X = pad_sequences(X) # padding our text vector so they all have the same length
X[:5]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,    35,  6114],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 22711,    15,    84,
           34,   887,   549],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 69802,     2,    37,    22,    14,    27,
          127,     5,  1722],
       [    0,     0,     0,     0,     0,     0,     0,  

## Model, Training and Testing

### Creating the model

In [8]:
!pip install --upgrade keras

Requirement already up-to-date: keras in /home/erolerten/anaconda3/envs/venv-tensorflow/lib/python3.7/site-packages (2.3.1)


In [9]:
config.gpu_options.per_process_gpu_memory_fraction = 0.95

model = Sequential()
model.add(Embedding(vocabulary_size, 256, input_length=X.shape[1]))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.5))
# model.add(Bidirectional(CuDNNGRU(256, return_sequences=True)))
model.add(AttentionLayer(name='attention',activation='sigmoid'))
model.add(BatchNormalization())
model.add(Dense(2, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))

Remark about Attention Layer: the imports in AttentionLayer module must be changed when tensorflow.keras.layers are being used.

#### Using Gradient Accumulation

In [10]:
STEPS = 5
# optimizer = keras.optimizers.Adam()
# optimizer = runai.ga.keras.optimizers.Optimizer(optimizer, STEPS)
optimizer = runai.ga.keras.optimizers.Adam(STEPS)

[Run:AI] [DEBUG   ] [12-04-2020 23:31:08.002] [1025] [optimizers.py          :16  ] Wrapping 'Adam' Keras optimizer with GA of 1 steps


#### Compile and show summary

In [11]:
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()
weight_matrix = model.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 39, 256)           25600000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 39, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 39, 512)           1050624   
_________________________________________________________________
dropout_2 (Dropout)          (None, 39, 512)           0         
_________________________________________________________________
attention (AttentionLayer)   (None, 512)               263168    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                

In [12]:
# for list in range(len(weight_matrix)):
#     weight_matrix[list].shape

In [13]:
# print(model.layers[3].trainable_weights)
# units = int(int(model.layers[3].trainable_weights[0].shape[1])/4)
# print("No units: ", units)

In [14]:
# W1 = model.layers[3].get_weights()[0]
# U1 = model.layers[3].get_weights()[1]
# b1 = model.layers[3].get_weights()[2]

# W1_i = W1[:, :units]
# W1_f = W1[:, units: units * 2]
# W1_c = W1[:, units * 2: units * 3]
# W1_o = W1[:, units * 3:]

# U1_i = U1[:, :units]
# U1_f = U1[:, units: units * 2]
# U1_c = U1[:, units * 2: units * 3]
# U1_o = U1[:, units * 3:]

# b1_i = b1[:units]
# b1_f = b1[units: units * 2]
# b1_c = b1[units * 2: units * 3]
# b1_o = b1[units * 3:]

In [15]:
# W2 = model.layers[4].get_weights()[0]
# U2 = model.layers[4].get_weights()[1]
# b2 = model.layers[4].get_weights()[2]

# W2_i = W2[:, :units]
# W2_f = W2[:, units: units * 2]
# W2_c = W2[:, units * 2: units * 3]
# W2_o = W2[:, units * 3:]

# U2_i = U2[:, :units]
# U2_f = U2[:, units: units * 2]
# U2_c = U2[:, units * 2: units * 3]
# U2_o = U2[:, units * 3:]

# b2_i = b2[:units]
# b2_f = b2[units: units * 2]
# b2_c = b2[units * 2: units * 3]
# b2_o = b2[units * 3:]

### Training the model

In [16]:
y = pd.get_dummies(df1['sentiment']).values
[print(df1['sentiment'][i], y[i]) for i in range(0,5)]

negative [1 0]
positive [0 1]
negative [1 0]
negative [1 0]
negative [1 0]


[None, None, None, None, None]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

#### Early stopping and model checkpoint mechanism

In [18]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss',patience=2)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


In [19]:
# tensorflow.compat.v1.disable_v2_behavior

#### Actual training

In [20]:
batch_size = 32
epochs = 30
import time
from datetime import datetime
datetime = str(datetime.now())
csv_logger = tf.keras.callbacks.CSVLogger('training'+datetime+'.log')
start = time.time()
print("started at:")
print(start)
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val,y_val), batch_size=batch_size, verbose=2, callbacks=[csv_logger,early_stop,model_checkpoint])
end = time.time()
elapsed = end - start
print(elapsed/60," minutes")

started at:
1586723472.8389375


TypeError: Cannot reconcile tf.cond 0-th outputs:
  branches returned: (<tensorflow.python.framework.indexed_slices.IndexedSlices object at 0x7f60740c0610>, <tf.Tensor 'training/Adam/runai/add:0' shape=(100000, 256) dtype=float32>)

Plotting training history

In [ ]:
# # Plot training & validation accuracy values
# plt.plot(history.history['accuracy'])
# # plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# # plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()


### Plotting training and validation accuracy and loss

In [ ]:
# plot train and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# plot train and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

### Saving the trained model

name of the file

In [ ]:
name = 'sentiment-analysis-trained-model'

In [ ]:
from datetime import datetime
dateTimeObj = datetime.now()
date = str(dateTimeObj.date())
time = str(dateTimeObj.time())
timestamp = date+time
punctuation = ['-',':','.']
for sign in punctuation:
    timestamp = timestamp.replace(sign,'')
print(timestamp)

In [ ]:
# model.save(name+timestamp+'.h5')

### Testing the model

In [ ]:
predictions = model.predict(X_test)
[print(X_test[i], predictions[i], y_test[i]) for i in range(0, 10)]

In [ ]:
accurate_prediction_count, inaccurate_prediction_count = 0, 0
for i, prediction in enumerate(predictions):
    if np.argmax(prediction)==np.argmax(y_test[i]):
        accurate_prediction_count += 1
    else:
        inaccurate_prediction_count += 1

total_predictions = accurate_prediction_count + inaccurate_prediction_count
print('Number of predictions: ', total_predictions)
print('Number of accurate predictions: ', accurate_prediction_count)
print('Number of false predictions: ', inaccurate_prediction_count)    
print('Accuracy: ', accurate_prediction_count/total_predictions)